In [32]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier , export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

preprocessed_df = pd.read_csv('/content/drive/MyDrive/DDos_dataset_fs.csv', index_col=0)
preprocessed_df.describe()

,act_data_pkt_fwd,Total Fwd Packets,Flow Bytes/s,Fwd Packet Length Max,Fwd Packet Length Std,Fwd Packet Length Mean,Total Backward Packets,Flow Duration,Bwd IAT Std,Bwd Packet Length Max,Fwd Packet Length Min,Bwd Packet Length Mean,Total Length of Bwd Packets,Total Length of Fwd Packets,Protocol,Label
count,424977.000000,424977.000000,4.249770e+05,424977.000000,424977.000000,424977.000000,424977.000000,4.249770e+05,4.249770e+05,424977.000000,424977.000000,424977.000000,4.249770e+05,424977.000000,424977.000000,424977.000000
mean,1.997181,4.097782,7.533926e+08,440.519943,8.751692,421.677894,1.546773,2.501030e+06,2.427182e+05,42.226600,415.918339,14.123509,1.100984e+03,1038.650035,14.448328,2.960525
std,30.884505,236.141021,9.933910e+08,472.547673,52.957962,459.229082,55.320571,1.424244e+07,1.831354e+06,294.361818,461.846397,92.229409,9.902007e+04,1816.955898,4.671059,1.969430
min,0.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1.000000,2.000000,7.166667e+05,211.000000,0.000000,76.173913,0.000000,1.000000e+00,0.000000e+00,0.000000,40.000000,0.000000,0.000000e+00,422.000000,17.000000,1.000000
50%,1.000000,2.000000,4.580000e+08,247.000000,0.000000,229.000000,0.000000,1.000000e+00,0.000000e+00,0.000000,229.000000,0.000000,0.000000e+00,494.000000,17.000000,3.000000
75%,1.000000,2.000000,9.140000e+08,481.000000,0.000000,457.000000,0.000000,4.900000e+01,0.000000e+00,0.000000,457.000000,0.000000,0.000000e+00,1322.000000,17.000000,5.000000
max,18766.000000,86743.000000,2.944000e+09,3625.000000,1448.583032,2131.000000,31700.000000,1.199975e+08,6.791569e+07,3627.000000,2131.000000,1850.769231,5.842950e+07,208524.000000,17.000000,7.000000


In [33]:
X = preprocessed_df.drop(' Label', axis=1)
y = preprocessed_df[" Label"]
y = y.replace(
    [1, 0, 2, 3, 4, 5, 6, 7],
    ['NetBIOS', 'BENIGN', 'MSSQL', 'LDAP', 'Portmap', 'Syn', 'UDP','UDPLag'])

In [34]:
clf = DecisionTreeClassifier(random_state=42 , max_depth=5)
clf.fit(X, y)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [35]:
from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(clf, out_file=None, 
                           feature_names=X.columns,  
                           class_names=y.unique(),  
                           filled=True, rounded=True,  
                           special_characters=True)

graph = graphviz.Source(dot_data)  
graph.render("DDos_decision_tree")  
graph.view()

'DDos_decision_tree.pdf'

In [36]:
rules = export_text(clf, feature_names=list(X.columns))
print(rules)

|---  Fwd Packet Length Min <= 401.50
|   |---  Fwd Packet Length Min <= 118.50
|   |   |---  Fwd Packet Length Max <= 19.50
|   |   |   |---  act_data_pkt_fwd <= 0.50
|   |   |   |   |---  Total Backward Packets <= 33.50
|   |   |   |   |   |--- class: BENIGN
|   |   |   |   |---  Total Backward Packets >  33.50
|   |   |   |   |   |--- class: Syn
|   |   |   |---  act_data_pkt_fwd >  0.50
|   |   |   |   |---  Fwd Packet Length Std <= 0.28
|   |   |   |   |   |--- class: Syn
|   |   |   |   |---  Fwd Packet Length Std >  0.28
|   |   |   |   |   |--- class: BENIGN
|   |   |---  Fwd Packet Length Max >  19.50
|   |   |   |---  Fwd Packet Length Min <= 93.50
|   |   |   |   |---  Fwd Packet Length Max <= 647.00
|   |   |   |   |   |--- class: BENIGN
|   |   |   |   |---  Fwd Packet Length Max >  647.00
|   |   |   |   |   |--- class: BENIGN
|   |   |   |---  Fwd Packet Length Min >  93.50
|   |   |   |   |---  Fwd Packet Length Min <= 116.50
|   |   |   |   |   |--- class: LDAP
|   |  

In [22]:
preprocessed_df = pd.read_csv('/content/drive/MyDrive/DDos_dataset.csv', index_col=0)
preprocessed_df

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
Protocol,,,,,,,,,,,,,,,,,,,,,
17,1,2,0,962.0,0.0,481.0,481.0,481.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
17,1,2,0,1174.0,0.0,587.0,587.0,587.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
17,1,2,0,1228.0,0.0,614.0,614.0,614.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
17,1,2,0,802.0,0.0,401.0,401.0,401.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
17,5,2,0,1488.0,0.0,744.0,744.0,744.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,26346,2,2,12.0,0.0,6.0,6.0,6.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
6,1058092,4,2,24.0,0.0,6.0,6.0,6.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
6,1053786,4,2,24.0,0.0,6.0,6.0,6.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
